# Generating CSV for earthquakes.

## Method:
1. Use twitter api to get data related to a #HashTag in a time period
2. Extract relevant data(magnitude, depth, date, time) from the text retrieved
3. Clean the data
4. Convert data to CSV


## 1. Use twitter api to get data related to a HashTag in a time period

1. After creating a developer account in twitter and applying for an academic licence we get the api_key and token to access the premium services of twitter api to get all the tweets related to a #hashtag

2. We search for the #Earthquakes to get all tweets related to earthquakes but for our experiment we foucus on reliable tweets therefore we have only fetched tweets from the USGS twitter page by searching for #USGS

3. We can fine-tune our search by including the date till which to search. Our period of search was 1.1.2008 to 1.1.2022

In [1]:
import tweepy
import pandas as pd
import re


# read configs
config = {}

api_key = "gU5n2ENwKk5ZitQhnFJD4LhEF"
api_key_secret = "dpqRaymjE1N3nHkQNA1k8cUo1GZtYvKoLARkBP0VjpnjymN9QW"

access_token = "941059418346602496-ppmslALuGBqeNKFEZaxpCazONh7dExI"
access_token_secret = "vX81wOWTg7hz486b5AbOftrMucN7Klx3buVjSZIqWfTTG"

# authentication
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

try:
    api = tweepy.API(auth)
    public_tweets = api.search_tweets("#USGS", until="2022-12-15")
except:
    print("API key, secret or token is missing or incorrect") 




## 2. Extract relevant data from the text retrieved

The text information contains a lot of information, much of which is unnecessary. Therefore we use regular expression matching to only retieve those results that are relevant to us. 

Example:
For earthquake we look for data such as magnitude, depth and date-time

In [2]:
def getData(text):
    m = re.search('Mag: (.*)\n', text)
    d = re.search('Depth: (.*)km\n', text)
    date = re.search('Date-Time: (.*)\n', text)
    l = re.search('M[0-9][.][0-9] - (.*)', text)
    
    magnitude=""
    depth=""
    dt=""
    location = ""
    
    if m:
        magnitude =  m.group(1)
        
    if d:
        depth = d.group(1)
        
    if date:
        dt = date.group(1)
        
    if l:
        location = l.group(1)
        
    
        
    return [magnitude, depth, dt, location]


## 3. Convert the data to a dataframe

After regular expression matching we have successfully extracted the relevant data.

We now combine the data we recieved from the twitter headers along with the data extracted from the tweet text and store it in a data frame.


In [3]:
# create dataframe
columns = ['DateTime', 'Time', 'Location', 'Type', 'Depth', 'Magnitude']
data = []
for tweet in public_tweets:
    reldata = getData(tweet.text)
    data.append([reldata[2], tweet.created_at, reldata[3], "Earthquake", reldata[1], reldata[0]])
    
df = pd.DataFrame(data, columns=columns)
df.dropna(subset=['DateTime'],inplace=True)

df

,DateTime,Time,Location,Type,Depth,Magnitude
0,,2022-12-14 23:59:30+00:00,,Earthquake,,
1,,2022-12-14 23:26:27+00:00,,Earthquake,,
2,,2022-12-14 23:07:20+00:00,,Earthquake,,
3,2022-12-13 02:05:10 UTC,2022-12-14 22:58:46+00:00,"Hindu Kush region, Afghanistan",Earthquake,10.015,4.3
4,2022-12-13 20:16:14 UTC,2022-12-14 22:53:16+00:00,"42 km ESE of Khorugh, Tajikistan",Earthquake,153.312,4.1
5,2022-12-12 02:11:07 UTC,2022-12-14 22:51:46+00:00,"30 km SSE of Jurm, Afghanistan",Earthquake,214.544,4.3
6,2022-12-11 09:53:38 UTC,2022-12-14 22:51:41+00:00,Afghanistan-Tajikistan-Pakistan region,Earthquake,98.531,4.7
7,2022-12-11 03:13:53 UTC,2022-12-14 22:51:24+00:00,"32 km SE of Jurm, Afghanistan",Earthquake,243.906,4.4
8,2022-12-11 01:52:42 UTC,2022-12-14 22:51:16+00:00,wMag: 4.2,Earthquake,41.477,4.2
9,2022-12-10 01:22:06 UTC,2022-12-14 22:48:58+00:00,,Earthquake,118.921,4


## 4. Convert data to CSV

We use one of python's built-in functions to convert a datafram to CSV file. This CSV file will be used for further processing later as mentioned in the other notebooks

In [4]:
df.to_csv('eathquake_tweets.csv')